# Tinder Analysis

In [23]:
'''
import pandas as pd
import numpy as np

profiles = pd.read_csv('tinder_dataset/profiles_2021-11-10.csv')
profiles

df = pd.read_json('tinder_dataset/profiles_2021-11-10.json')
df'''

"\nimport pandas as pd\nimport numpy as np\n\nprofiles = pd.read_csv('tinder_dataset/profiles_2021-11-10.csv')\nprofiles\n\ndf = pd.read_json('tinder_dataset/profiles_2021-11-10.json')\ndf"

# Mongo DB

In [22]:
import json
import pymongo
from pymongo import MongoClient

client = MongoClient('localhost', 27017, username='mongoadmin', password='pass1234')

# Get the list of DBs already defined
print(client.list_database_names())

# Create a new DB - bda
database = client['bda']

# Create a collection
adults = database.adults

# Load the data
with open('tinder_dataset/profiles_2021-11-10.json', 'r') as f:
    data = json.load(f)

['admin', 'config', 'local']


In [ ]:
data

In [ ]:
type(data)

In [24]:
# Load the data
adults_list = list(data.values())
result = adults.insert_many(adults_list)
adults.count_documents({})

AttributeError: 'list' object has no attribute 'values'

In [ ]:
# Check if data was loaded correctly
adults.find_one()